## Content analysis stuff

In [3]:
from nlp_functions import * 
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline
# Example usage:
updated_df = extract_named_entities_hf_split("test-data\Bellingcat-regions-india-subcontinent.json.csv", 'content')
updated_df.to_csv('ind_channel.csv', index=False)

<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
C:\Users\User\AppData\Local\Temp\ipykernel_56792\234342389.py:5: SyntaxWarning: invalid escape sequence '\B'
  updated_df = extract_named_entities_hf_split("test-data\Bellingcat-regions-india-subcontinent.json.csv", 'content')
100%|██████████| 1227/1227 [01:32<00:00, 13.23it/s]


In [4]:
file_path = 'ind_channel.csv'
column_name = 'content'
output_file_path = 'output_file_ncs3.csv'

In [5]:
# Noun phrase exraction (individual table)
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from textblob import TextBlob

# Function to extract noun phrases
def extract_noun_phrases(text):
    blob = TextBlob(text)
    return [np for np in blob.noun_phrases]

# Function to process CSV and extract noun phrases
def extract_noun_chunks_from_csv1(file_path, column_name):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the CSV file.")
    
    # Extract text from the specified column
    texts = df[column_name].dropna().tolist()

    # Initialize a list to store the noun chunks
    all_noun_chunks = []

    # Process each text entry
    for text in texts:
        noun_chunks = extract_noun_phrases(text)
        all_noun_chunks.append(noun_chunks)
    
    # Create a new DataFrame with the extracted noun chunks
    result_df = pd.DataFrame({
        column_name: texts,
        'noun_chunks': all_noun_chunks
    })

    return result_df

# Example usage:
result_df = extract_noun_chunks_from_csv1(file_path, column_name)
result_df.head(20)


,content,noun_chunks
0,Naxanites are funded by Pakistan even though P...,"[naxanites, pakistan, pakistan, hates commies,..."
1,Naxalites are struggling to survive in the for...,"[naxalites, indian government]"
2,Tru tru,[tru]
3,There is always ten others to take their place...,[]
4,"Haha, I mean India is not the paradigm of a st...","[haha, india, stable country]"
5,Well at least it is better than Pakistan,"[well, pakistan]"
6,They take the meat as one of two countries tha...,[]
7,The other country that does that is Iran,[iran]
8,> They take the meat as one of two countries t...,[max pigeon nasty]
9,"Staff , Is posting IMINT/OSINT tweets allowed ...","[staff, imint/osint]"


In [6]:
# Function to extract noun phrases
def extract_noun_phrases(text):
    blob = TextBlob(text)
    return [np for np in blob.noun_phrases]

# Function to process CSV and extract noun phrases
def extract_noun_chunks_from_csv(file_path, column_name, output_file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the CSV file.")
    
    # Extract text from the specified column
    texts = df[column_name].dropna().tolist()

    # Initialize a list to store the noun chunks
    all_noun_chunks = []

    # Process each text entry
    for text in texts:
        noun_chunks = extract_noun_phrases(text)
        noun_chunks_str = ', '.join(noun_chunks)  # Convert list to comma-separated string
        all_noun_chunks.append(noun_chunks_str)
    
    # Append noun_chunks to the original DataFrame
    df['noun_chunks'] = pd.Series(all_noun_chunks)

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file_path, index=False)

# Example usage:
extract_noun_chunks_from_csv(file_path, column_name, output_file_path)
print(f"Updated CSV file saved to {output_file_path}")




Updated CSV file saved to output_file_ncs3.csv


In [ ]:
# Count noun phrases 
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords

# Define custom stopwords
custom_stopwords = [
    'i', 'im', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself',
    'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself',
    'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that',
    'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as',
    'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
    'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',
    'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should',
    'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn',
    'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn',
    'm', 'dms', 'nytimes', 'bbc', 'apnews', 'outlookindia','two','10','000','##'
]

# Load the stopwords from NLTK
nltk_stopwords = set(stopwords.words('english'))
# Combine NLTK stopwords with custom stopwords
stop_words = nltk_stopwords.union(set(custom_stopwords))

def is_valid_phrase(phrase):
    # Split the phrase into words
    words = phrase.split()
    # Check if the phrase contains any non-stopword words
    return any(word.lower() not in stop_words for word in words)

def count_noun_phrases(file_path, column_name):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Extract noun phrases from the specified column
    noun_phrases = df[column_name].dropna().str.split(', ')

    # Flatten the list of noun phrases
    all_noun_phrases = [phrase for sublist in noun_phrases for phrase in sublist]

    # Filter out phrases that are empty or contain only stopwords
    valid_noun_phrases = [phrase for phrase in all_noun_phrases if is_valid_phrase(phrase)]

    # Count the frequency of each valid noun phrase
    noun_phrase_counts = Counter(valid_noun_phrases)

    # Convert to a DataFrame for better visualization
    noun_phrase_counts_df = pd.DataFrame(noun_phrase_counts.items(), columns=['Noun Phrase', 'Frequency'])
    noun_phrase_counts_df = noun_phrase_counts_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

    return noun_phrase_counts_df

# Example usage:
file_path = 'output_file_ncs3.csv'
column_name = 'noun_chunks'
noun_phrase_counts_df = count_noun_phrases(file_path, column_name)



#### Entity plots

In [10]:
import pandas as pd
import plotly.express as px
import nlp_functions as nf
nf.plot_top_entities(file_path, ['Organization','Person'], top_n=25)
nf.plot_top_entities(file_path, ['Location'], top_n=25)
nf.plot_top_entities(file_path, ['Weapon'], top_n=40)

### Posts over time 

In [20]:
data = pd.read_csv(file_path)
nf.filter_and_plot(data,aggregation='weekly')
nf.filter_and_plot(data,regex='Weapon',aggregation='weekly',column='entity_types')
nf.filter_and_plot(data,regex='Location',aggregation='weekly',column='entity_types')
nf.filter_and_plot(data,regex='Organisation',aggregation='weekly',column='entity_types')

c:\Users\User\Desktop\uni-CA\nlp_functions.py:289: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

c:\Users\User\Desktop\uni-CA\nlp_functions.py:289: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



c:\Users\User\Desktop\uni-CA\nlp_functions.py:289: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



c:\Users\User\Desktop\uni-CA\nlp_functions.py:289: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



c:\Users\User\Desktop\uni-CA\nlp_functions.py:289: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



## Model testing space

In [ ]:
# Load the model and tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "eventdata-utd/conflibert-named-entity-recognition"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Sample text
text = "John Doe is a software engineer at OpenAI, and he lives in San Francisco."

# Perform NER
ner_results = ner_pipeline(text)

# Print the results
for entity in ner_results:
    print(entity)
    print(f"Entity: {entity['word']}, Label: {entity['entity']}")
